In [2]:
import numpy as np
import pandas as pd
import math

import common
from common import *

In [3]:
occurrence_probabilities = [0.15, 0.1, 0.05, 0.3, 0.25, 0.15]
codes = ["000000", "001001", "001110", "011011", "111001", "111111"]
pe = 0.1

In [4]:
distance_matrix = build_distance_matrix(codes)
np.array(distance_matrix)

array([[0, 2, 3, 4, 4, 6],
       [2, 0, 3, 2, 2, 4],
       [3, 3, 0, 3, 5, 3],
       [4, 2, 3, 0, 2, 2],
       [4, 2, 5, 2, 0, 2],
       [6, 4, 3, 2, 2, 0]])

In [5]:
channel_matrix = build_channel_matrix(codes, pe)
np.array(channel_matrix)

array([[9.92547e-01, 6.56100e-03, 7.29000e-04, 8.10000e-05, 8.10000e-05,
        1.00000e-06],
       [6.56100e-03, 9.79507e-01, 7.29000e-04, 6.56100e-03, 6.56100e-03,
        8.10000e-05],
       [7.29000e-04, 7.29000e-04, 9.97075e-01, 7.29000e-04, 9.00000e-06,
        7.29000e-04],
       [8.10000e-05, 6.56100e-03, 7.29000e-04, 9.79507e-01, 6.56100e-03,
        6.56100e-03],
       [8.10000e-05, 6.56100e-03, 9.00000e-06, 6.56100e-03, 9.80227e-01,
        6.56100e-03],
       [1.00000e-06, 8.10000e-05, 7.29000e-04, 6.56100e-03, 6.56100e-03,
        9.86067e-01]])

In [6]:
undetected_error_probabilities = get_undetected_error_probabilities(channel_matrix)
np.array(undetected_error_probabilities)

array([0.007453, 0.020493, 0.002925, 0.020493, 0.019773, 0.013933])

In [7]:
undetected_error_over_occurrence_probabilities = np.multiply(undetected_error_probabilities, occurrence_probabilities)
undetected_error_over_occurrence_probabilities

array([0.00111795, 0.0020493 , 0.00014625, 0.0061479 , 0.00494325,
       0.00208995])

In [8]:
undetected_error_probability = sum(undetected_error_over_occurrence_probabilities)
undetected_error_probability

0.016494600000000005

Побудуємо допоміжну матрицю для розрахунку загальної умовної ентропії.

In [9]:
conditional_entropy_matrix = build_conditional_entropy_matrix(channel_matrix)
conditional_entropy_matrix

array([[1.07122374e-02, 4.75795096e-02, 7.59748751e-03, 1.10092920e-03,
        1.10092920e-03, 1.99315686e-05],
       [4.75795096e-02, 2.92601193e-02, 7.59748751e-03, 4.75795096e-02,
        4.75795096e-02, 1.10092920e-03],
       [7.59748751e-03, 7.59748751e-03, 4.21370539e-03, 7.59748751e-03,
        1.50854792e-04, 7.59748751e-03],
       [1.10092920e-03, 4.75795096e-02, 7.59748751e-03, 2.92601193e-02,
        4.75795096e-02, 4.75795096e-02],
       [1.10092920e-03, 4.75795096e-02, 1.50854792e-04, 4.75795096e-02,
        2.82425053e-02, 4.75795096e-02],
       [1.99315686e-05, 1.10092920e-03, 7.59748751e-03, 4.75795096e-02,
        4.75795096e-02, 1.99603810e-02]])

In [10]:
entropy_sums = np.sum(conditional_entropy_matrix, axis=1)
entropy_sums

array([0.06811102, 0.18069706, 0.03475451, 0.18069706, 0.17223282,
       0.12383775])

Загальна умовна ентропія $H(B/А)$

In [11]:
entropy_over_occurrence = np.multiply(occurrence_probabilities, entropy_sums)
total_conditional_entropy = sum(entropy_over_occurrence)
total_conditional_entropy

0.14586707200681168

Мінімізація кодового відображення за ймовірністю невиявлення помилок

In [12]:
final_table = {"symbol": [f"a_{i}" for i in range(len(codes))],
               "code": codes,
               "P(ai)": occurrence_probabilities,
               "Pнп(ai)": undetected_error_probabilities,
               "P(ai)Pнп(ai)": undetected_error_over_occurrence_probabilities,
               "entropy": entropy_sums,
               "entropy_over_occurrence": entropy_over_occurrence}
final_df = pd.DataFrame(final_table)
final_df

,symbol,code,P(ai),Pнп(ai),P(ai)Pнп(ai),entropy,entropy_over_occurrence
0,a_0,000000,0.15,0.007453,0.001118,0.068111,0.010217
1,a_1,001001,0.10,0.020493,0.002049,0.180697,0.018070
2,a_2,001110,0.05,0.002925,0.000146,0.034755,0.001738
3,a_3,011011,0.30,0.020493,0.006148,0.180697,0.054209
4,a_4,111001,0.25,0.019773,0.004943,0.172233,0.043058
5,a_5,111111,0.15,0.013933,0.002090,0.123838,0.018576


In [13]:
# Sort by Column1 in ascending order and by Column2 in descending order
min_df = final_df.sort_values(by=["Pнп(ai)"], ascending=[False ])
min_df

,symbol,code,P(ai),Pнп(ai),P(ai)Pнп(ai),entropy,entropy_over_occurrence
1,a_1,001001,0.10,0.020493,0.002049,0.180697,0.018070
3,a_3,011011,0.30,0.020493,0.006148,0.180697,0.054209
4,a_4,111001,0.25,0.019773,0.004943,0.172233,0.043058
5,a_5,111111,0.15,0.013933,0.002090,0.123838,0.018576
0,a_0,000000,0.15,0.007453,0.001118,0.068111,0.010217
2,a_2,001110,0.05,0.002925,0.000146,0.034755,0.001738


In [14]:
min_codes = min_df["code"].tolist()
distance_matrix_min = build_distance_matrix(min_codes)
np.array(distance_matrix_min)

array([[0, 2, 2, 4, 2, 3],
       [2, 0, 2, 2, 4, 3],
       [2, 2, 0, 2, 4, 5],
       [4, 2, 2, 0, 6, 3],
       [2, 4, 4, 6, 0, 3],
       [3, 3, 5, 3, 3, 0]])

In [15]:
min_codes

['001001', '011011', '111001', '111111', '000000', '001110']

In [16]:
channel_matrix_min = build_channel_matrix(min_codes, pe)
np.array(channel_matrix_min)

array([[9.79507e-01, 6.56100e-03, 6.56100e-03, 8.10000e-05, 6.56100e-03,
        7.29000e-04],
       [6.56100e-03, 9.79507e-01, 6.56100e-03, 6.56100e-03, 8.10000e-05,
        7.29000e-04],
       [6.56100e-03, 6.56100e-03, 9.80227e-01, 6.56100e-03, 8.10000e-05,
        9.00000e-06],
       [8.10000e-05, 6.56100e-03, 6.56100e-03, 9.86067e-01, 1.00000e-06,
        7.29000e-04],
       [6.56100e-03, 8.10000e-05, 8.10000e-05, 1.00000e-06, 9.92547e-01,
        7.29000e-04],
       [7.29000e-04, 7.29000e-04, 9.00000e-06, 7.29000e-04, 7.29000e-04,
        9.97075e-01]])

In [17]:
undetected_error_probabilities_min = get_undetected_error_probabilities(channel_matrix_min)
np.array(undetected_error_probabilities_min)
error_probabilities_min = np.multiply(undetected_error_probabilities_min, occurrence_probabilities)
error_probabilities_min

array([0.00307395, 0.0020493 , 0.00098865, 0.0041799 , 0.00186325,
       0.00043875])

In [18]:
undetected_error_probability_min = sum(error_probabilities_min)
undetected_error_probability-undetected_error_probability_min

0.0039008000000000115